In [ ]:
# Copyright 2022 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# GCP上的E2E ML：MLOps阶段6：开始使用自定义预测例程（CPR）

<table align="left">
  <td>
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/community/ml_ops/stage6/get_started_with_cpr.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> 在Colab中运行
    </a>
  </td>
  <td>
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/community/ml_ops/stage6/get_started_with_cpr.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      在GitHub上查看
    </a>
  </td>
  <td>
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/vertex-ai-samples/main/notebooks/community/ml_ops/stage6/get_started_with_cpr.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo">
      在Vertex AI Workbench中打开
    </a>
  </td>         
</table>
<br/><br/><br/>

## 概述

本教程演示如何使用 Vertex AI SDK 构建一个自定义容器，使用 Custom Prediction Routine 模型服务器来为 Vertex AI Predictions 服务 scikit-learn 模型。这是目前一个**试验性**功能，并且目前还未得到 Vertex AI SDK 的官方支持。在本教程中，您将会从 github 的一个实验性分支安装 Vertex AI SDK。

### 目标

在本教程中，您将学习如何在`Vertex AI Predictions`中使用自定义预测例程（CPR）。

本教程使用以下谷歌云ML服务：

- `Vertex AI Training`
- `Vertex AI Predictions`
- `Vertex AI Custom Predictions`
- `Google Artifact Registry`

执行的步骤包括：

- 编写自定义数据预处理器。
- 训练模型。
- 使用自定义数据预处理的自定义scikit-learn服务容器构建，使用自定义预测例程模型服务器。
    - 在本地测试模型服务容器。
    - 上传并部署模型服务容器到Vertex AI Endpoint。
    - 发出预测请求。
- 使用自定义预测器（后处理）构建自定义scikit-learn服务容器，使用自定义预测例程模型服务器。
    - 实现自定义预测器。
    - 在本地测试模型服务容器。
    - 上传并部署模型服务容器到Vertex AI Endpoint。
    - 发出预测请求。
- 使用自定义预测器和HTTP请求处理程序构建自定义scikit-learn服务容器，使用自定义预测例程模型服务器。
    - 实现自定义处理程序。
    - 在本地测试模型服务容器。
    - 上传并部署模型服务容器到Vertex AI Endpoint。
    - 发出预测请求。
- 为自定义scikit-learn服务容器的Dockerfile进行自定义，使用自定义预测例程模型服务器的自定义预测器和HTTP请求处理程序。
    - 实现自定义Dockerfile。
    - 在本地测试模型服务容器。
    - 上传并部署模型服务容器到Vertex AI Endpoint。
    - 发出预测请求。

### 数据集

这个教程使用的数据集是来自[TensorFlow Datasets](https://www.tensorflow.org/datasets/catalog/overview)的[Iris数据集](https://www.tensorflow.org/datasets/catalog/iris)。这个数据集不需要任何特征工程。本教程中的数据集版本存储在一个公共的云存储桶中。训练好的模型可以预测三种不同种类的鸢尾花：山鸢尾（setosa）、维吉尼亚鸢尾（virginica）或鸢尾花（versicolor）。

### 成本
本教程使用谷歌云的计费组件：

- Vertex AI
- 云存储

请了解[Vertex AI定价](https://cloud.google.com/vertex-ai/pricing)和[云存储定价](https://cloud.google.com/storage/pricing)，并使用[定价计算器](https://cloud.google.com/products/calculator/)根据您预计的使用量生成成本估算。

## 安装

安装额外的软件包依赖项，这些软件包在您的笔记本环境中尚未安装，例如NumPy、Scikit-learn、FastAPI、Uvicorn和joblib。请使用每个软件包的最新主要GA版本。

In [ ]:
! mkdir src

In [ ]:
%%writefile src/requirements.txt
fastapi
uvicorn
joblib~=1.0
numpy~=1.20
scikit-learn~=0.24

In [ ]:
import os

# The Vertex AI Workbench Notebook product has specific requirements
IS_WORKBENCH_NOTEBOOK = os.getenv("DL_ANACONDA_HOME")
IS_USER_MANAGED_WORKBENCH_NOTEBOOK = os.path.exists(
    "/opt/deeplearning/metadata/env_version"
)

# Vertex AI Notebook requires dependencies to be installed with '--user'
USER_FLAG = ""
if IS_WORKBENCH_NOTEBOOK:
    USER_FLAG = "--user"


! pip3 install --upgrade google-cloud-aiplatform[prediction] {USER_FLAG} -q
! pip3 install --upgrade google-cloud-storage {USER_FLAG} -q

# Install the same dependencies used in the serving container in the notebook
# environment.
! pip3 install -U {USER_FLAG} -r src/requirements.txt

重新启动内核

在安装附加包后，您需要重新启动笔记本内核，以便它能够找到这些包。

In [ ]:
# Automatically restart kernel after installs
import os

if not os.getenv("IS_TESTING"):
    # Automatically restart kernel after installs
    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

## 在开始之前

### GPU 运行时

*如果有此选项，请确保在GPU运行时中运行此笔记本。在Colab中，选择* **Runtime > Change Runtime Type > GPU**

### 设置您的Google Cloud项目

**无论您的笔记本环境如何，以下步骤都是必需的。**

1. [选择或创建一个Google Cloud项目](https://console.cloud.google.com/cloud-resource-manager)。当您首次创建帐户时，您将获得300美元的免费信用额用于计算/存储成本。

2. [确保为您的项目启用了计费。](https://cloud.google.com/billing/docs/how-to/modify-project)

3. [启用以下API：Vertex AI API、Compute Engine API和云存储。](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com,compute_component,storage-component.googleapis.com)

4. 如果您是在本地运行此笔记本，您需要安装[Cloud SDK]((https://cloud.google.com/sdk)。

5. 在下面的单元格中输入您的项目ID。然后运行该单元格，以确保Cloud SDK在本笔记本中的所有命令中使用正确的项目。

**注意**：Jupyter会将以`!`为前缀的行作为shell命令运行，并会对以`$`为前缀的Python变量进行插值。

设置您的项目ID

**如果您不知道您的项目ID**，您可以使用 `gcloud` 来获取您的项目ID。

In [ ]:
PROJECT_ID = "[your-project-id]"  # @param {type:"string"}

In [ ]:
if PROJECT_ID == "" or PROJECT_ID is None or PROJECT_ID == "[your-project-id]":
    # Get your GCP project id from gcloud
    shell_output = ! gcloud config list --format 'value(core.project)' 2>/dev/null
    PROJECT_ID = shell_output[0]
    print("Project ID:", PROJECT_ID)

In [ ]:
! gcloud config set project $PROJECT_ID

区域

您还可以更改“REGION”变量，该变量用于本笔记本其余部分的操作。 以下是 Vertex AI 支持的区域。我们建议您选择最接近您的区域。

- 美洲：`us-central1`
- 欧洲：`europe-west4`
- 亚太：`asia-east1`

您可能不会使用多区域存储桶来进行 Vertex AI 的训练或预测。并非所有区域都支持所有 Vertex AI 服务。

了解更多关于 [Vertex AI 区域](https://cloud.google.com/vertex-ai/docs/general/locations)。

In [ ]:
REGION = "[your-region]"  # @param {type: "string"}

if REGION == "[your-region]":
    REGION = "us-central1"

时间戳

如果您正在参加实时教程会话，您可能正在使用共享的测试帐户或项目。为了避免用户在创建的资源上出现命名冲突，您需要为每个实例会话创建一个时间戳，并将时间戳附加到您在本教程中创建的资源名称中。

In [ ]:
from datetime import datetime

TIMESTAMP = datetime.now().strftime("%Y%m%d%H%M%S")

###验证您的Google Cloud帐户

**如果您正在使用Vertex AI Workbench笔记本**，您的环境已经得到了验证。跳过此步骤。

**如果您正在使用Colab**，运行下面的单元格，并按照提示进行身份验证，通过oAuth认证您的帐户。

**否则**，请按照以下步骤操作：

在Cloud控制台中，转到[创建服务帐户密钥](https://console.cloud.google.com/apis/credentials/serviceaccountkey)页面。

**单击创建服务帐户**。

在**服务帐户名称**字段中输入一个名称，然后单击**创建**。

在**为该服务帐户授予此项目的访问权限**部分，单击**角色**下拉列表。在筛选框中输入“Vertex”，然后选择**Vertex管理员**。在筛选框中输入“存储对象管理员”，然后选择**存储对象管理员**。

单击创建。包含您密钥的JSON文件将下载到您的本地环境中。

在下面的单元格中，将您的服务帐户密钥路径输入为GOOGLE_APPLICATION_CREDENTIALS变量并运行该单元格。

In [ ]:
# If you are running this notebook in Colab, run this cell and follow the
# instructions to authenticate your GCP account. This provides access to your
# Cloud Storage bucket and lets you submit training jobs and prediction
# requests.

import os
import sys

# If on Vertex AI Workbench, then don't execute this code
IS_COLAB = False
if not os.path.exists("/opt/deeplearning/metadata/env_version") and not os.getenv(
    "DL_ANACONDA_HOME"
):
    if "google.colab" in sys.modules:
        IS_COLAB = True
        from google.colab import auth as google_auth

        google_auth.authenticate_user()

    # If you are running this notebook locally, replace the string below with the
    # path to your service account key and run this cell to authenticate your GCP
    # account.
    elif not os.getenv("IS_TESTING"):
        %env GOOGLE_APPLICATION_CREDENTIALS ''

### 创建一个云存储桶

**无论您使用什么笔记本环境，都需要执行以下步骤。**

当您使用Python初始化Vertex AI SDK时，需要指定一个云存储暂存桶。这个暂存桶是您数据集和模型资源的关联数据在不同会话中保留的地方。

在下面设置您的云存储桶的名称。存储桶的名称必须在所有谷歌云项目中全局唯一，包括您组织以外的项目。

In [ ]:
BUCKET_NAME = "[your-bucket-name]"  # @param {type:"string"}
BUCKET_URI = f"gs://{BUCKET_NAME}"

In [ ]:
if BUCKET_NAME == "" or BUCKET_NAME is None or BUCKET_NAME == "[your-bucket-name]":
    BUCKET_NAME = PROJECT_ID + "aip-" + TIMESTAMP
    BUCKET_URI = "gs://" + BUCKET_NAME

只有在您的存储桶尚不存在时才运行以下单元格以创建您的云存储桶。

In [ ]:
! gsutil mb -l $REGION $BUCKET_URI

最后，通过检查云存储桶的内容来验证对其的访问权限。

In [ ]:
! gsutil ls -al $BUCKET_URI

### 设置变量

接下来，设置一些在整个教程中使用的变量。
### 导入库并定义常量

In [ ]:
import google.cloud.aiplatform as aip

### 初始化 Python 的 Vertex AI SDK

为您的项目和相应的存储桶初始化 Python 的 Vertex AI SDK。

In [ ]:
aip.init(project=PROJECT_ID, staging_bucket=BUCKET_URI)

## 编写自定义数据预处理

首先，为训练数据编写名为`preprocess.py`的模块进行数据预处理。由于所有特征都是数值型的，每个特征列都将被标准化 - 即均值为0，标准差为1。这也被称为对数值特征值进行缩放。

In [ ]:
%%writefile src/preprocess.py
import numpy as np

class MySimpleScaler(object):
    def __init__(self):
        self._means = None
        self._stds = None

    def preprocess(self, data):
        if self._means is None:  # during training only
            self._means = np.mean(data, axis=0)

        if self._stds is None:  # during training only
            self._stds = np.std(data, axis=0)
            if not self._stds.all():
                raise ValueError("At least one column has standard deviation of 0.")

        return (data - self._means) / self._stds

## 训练并存储模型和数据预处理模块

接下来，按照以下步骤训练模型：

1. 使用 `preprocess.MySimpleScaler` 对鸢尾花数据进行预处理。
2. 使用 scikit-learn 训练模型。
3. 将训练好的模型导出为 joblib（.joblib）文件。
4. 将你的 `MySimpleScaler` 实例导出为 pickle（.pkl）文件。

In [ ]:
%mkdir model

In [ ]:
%cd src

import pickle

import joblib
from preprocess import MySimpleScaler
from sklearn.datasets import load_iris
from sklearn.ensemble import RandomForestClassifier

iris = load_iris()
scaler = MySimpleScaler()

X = scaler.preprocess(iris.data)
y = iris.target

model = RandomForestClassifier()
model.fit(X, y)

joblib.dump(model, "../model/model.joblib")
with open("../model/preprocessor.pkl", "wb") as f:
    pickle.dump(scaler, f)

%cd ..

### 上传模型工件和自定义数据预处理器到云存储

要部署您的模型，模型工件 `model.joblib` 和数据预处理器 `preprocessor.pkl` 需要存储在云存储中。

In [ ]:
! gsutil cp model/* {BUCKET_URI}/model/
! gsutil ls {BUCKET_URI}/model/

使用CPR模型服务器构建自定义模型服务容器的过程：场景1：实现预处理器和后处理器

接下来，是时候为训练好的模型和数据预处理器构建一个自定义的服务容器了。对于数据预处理，我们可以通过使用AutoGraph编译器（例如，@tf.function装饰器）将数据预处理器与模型合并，将Python代码转换为静态图。这种方法有以下几个限制：

- 不是所有的Python操作都可以转换为图操作。
- 仅支持静态图操作。

虽然这种简单的数据预处理器可以转换为静态图，但许多更复杂的预处理和后处理无法这样做。在这种情况下，我们希望预处理和后处理步骤作为纯Python代码执行，其中：

- 数据预处理在HTTP服务器和模型输入之间被插入。
- 数据预处理被沙盒化，这样如果引发异常，不会导致模型服务器崩溃。

Vertex AI自定义预测例程提供了一个模板方法来实现上述操作，可以直接使用。

了解有关[自定义预测例程模型服务器](https://github.com/googleapis/python-aiplatform/blob/custom-prediction-routine/google/cloud/aiplatform/prediction/model_server.py)的更多信息。

自定义模型服务容器包含以下三个代码组件：

1. [模型服务器](https://github.com/googleapis/python-aiplatform/blob/custom-prediction-routine/google/cloud/aiplatform/prediction/model_server.py)
   - 托管模型的HTTP服务器
   - 负责设置路由/端口等
   - 在这个示例中，我们将直接使用`google.cloud.aiplatform.prediction.model_server.ModelServer`。
2. [请求处理器](https://github.com/googleapis/python-aiplatform/blob/custom-prediction-routine/google/cloud/aiplatform/prediction/handler.py)
   - 负责处理请求的网络服务器方面，例如反序列化请求主体，序列化响应，设置响应头等。
   - 在这个示例中，我们将使用SDK中提供的默认处理器`google.cloud.aiplatform.prediction.handler.PredictionHandler`。
3. [预测器](https://github.com/googleapis/python-aiplatform/blob/custom-prediction-routine/google/cloud/aiplatform/prediction/predictor.py)
   - 负责处理预测请求的ML逻辑。

这三个组件中的每一个都可以根据自定义容器的要求进行定制。

您可以使用预定义的[`SklearnPredictor`](https://github.com/googleapis/python-aiplatform/blob/custom-prediction-routine/google/cloud/aiplatform/prediction/sklearn/predictor.py)作为您的`CprPredictor`的基类。您只需要实现`load`，`preprocess`和`postprocess`方法。

请先实现一个自定义的`Predictor`，加载预处理器。然后在`preprocess`时使用这个预处理器。

In [ ]:
%%writefile src/predictor.py

import joblib
import numpy as np
import pickle

from google.cloud import storage
from google.cloud.aiplatform.prediction.sklearn.predictor import SklearnPredictor

from sklearn.datasets import load_iris


class CprPredictor(SklearnPredictor):
    
    def __init__(self):
        return
    
    def load(self, gcs_artifacts_uri: str):
        """Loads the preprocessor artifacts."""
        super().load(gcs_artifacts_uri)
        gcs_client = storage.Client()
        with open("preprocessor.pkl", 'wb') as preprocessor_f:
            gcs_client.download_blob_to_file(
                f"{gcs_artifacts_uri}/preprocessor.pkl", preprocessor_f
            )

        with open("preprocessor.pkl", "rb") as f:
            preprocessor = pickle.load(f)

        self._class_names = load_iris().target_names
        self._preprocessor = preprocessor
    
    def preprocess(self, prediction_input):
        """Perform scaling preprocessing"""
        inputs = super().preprocess(prediction_input)
        return self._preprocessor.preprocess(inputs)
    
    def postprocess(self, prediction_results):
        """Convert class indices to class names."""
        return {"predictions": [self._class_names[class_num] for class_num in prediction_results]}

构建并推送容器到Artifact Registry.

### 构建您的容器

要构建一个自定义容器，我们还需要编写一个启动模型服务器的镜像入口点。然而，使用自定义预测例程功能时，您不再需要编写入口点。Vertex AI SDK 将使用您提供的自定义预测器填充入口点。

#### 设置凭据（用于本地执行）

设置凭据仅在本地运行自定义服务容器时才需要。设置凭据是为了执行“Predictor”的“load”函数，该函数从云存储下载模型工件。

根据为您的服务账号授予的权限，有两种设置凭据的选项。

如果尚未启用IAM API，请首先启用它。

In [ ]:
! gcloud services enable iam.googleapis.com

#### 选项1：服务帐号

按照以下步骤操作：

1. 在 Cloud 控制台中，转到 [**创建服务帐号密钥**页面](https://console.cloud.google.com/apis/credentials/serviceaccountkey)。

2. 点击 **创建服务帐号**。

3. 在 **服务帐号名称**字段中输入名称，然后点击 **创建**。

4. 在 **授予此服务帐号访问项目**部分，点击 **角色** 下拉列表。在过滤框中输入 "Vertex AI"，然后选择 **Vertex AI 管理员**。在过滤框中输入 "存储对象管理员"，然后选择 **存储对象管理员**。

接下来，生成服务帐号密钥，并将其保存到您运行此笔记本的同一目录中的 `credentials.json` 文件中。

In [ ]:
SERVICE_ACCOUNT = "[your-service-account]"  # @param {type:"string"}

In [ ]:
if (
    SERVICE_ACCOUNT == ""
    or SERVICE_ACCOUNT is None
    or SERVICE_ACCOUNT == "[your-service-account]"
):
    # Get your GCP project id from gcloud
    shell_output = !gcloud auth list 2>/dev/null
    SERVICE_ACCOUNT = shell_output[2].replace("*", "").strip()
    print("Service Account:", SERVICE_ACCOUNT)

In [ ]:
! gcloud iam service-accounts keys create credentials.json --iam-account=$SERVICE_ACCOUNT
! gcloud auth application-default login

选项2：用户帐户

按照以下步骤操作：

1. 打开终端并cd到运行笔记本的相同目录。

2. 执行命令 `gcloud auth application-default login` -- 答复yes以继续。这将打开一个身份验证浏览器标签页。按照说明操作。

In [ ]:
CREDENTIALS_FILE = "/home/jupyter/.config/gcloud/application_default_credentials.json"

构建您的自定义模型服务容器

构建自定义图像需要一个Dockerfile，您需要在其中实现图像的外观。使用自定义预测例程功能，Vertex AI SDK会自动生成Dockerfile并为您构建图像。

默认情况下，使用`python:3.7`作为基础图像。

In [ ]:
import os

from google.cloud.aiplatform.prediction import LocalModel
from src.predictor import CprPredictor

REPOSITORY = "custom-preprocess-container-prediction"  # @param {type:"string"}
SERVER_IMAGE = "sklearn-cpr-preprocess-server"  # @param {type:"string"}

local_model = LocalModel.build_cpr_model(
    "src",
    f"{REGION}-docker.pkg.dev/{PROJECT_ID}/{REPOSITORY}/{SERVER_IMAGE}",
    predictor=CprPredictor,
    requirements_path="src/requirements.txt",
)

获取用于服务的容器规格

接下来，显示您刚刚构建的定制服务容器的规格。

In [ ]:
local_model.get_serving_container_spec()

### 创建示例数据

接下来，创建一些合成的示例数据，并将这些示例数据以 JSON 格式存储用于预测。

了解更多关于 [在 JSON 中格式化输入实例](https://cloud.google.com/vertex-ai/docs/predictions/online-predictions-custom-models#request-body-details) 的信息。

In [ ]:
INPUT_FILE = "instances.json"

In [ ]:
%%writefile $INPUT_FILE
{
    "instances": [
        [6.7, 3.1, 4.7, 1.5],
        [4.6, 3.1, 1.5, 0.2]
    ]
}

### 在本地测试自定义模型服务容器

接下来，您可以在本地测试您的自定义模型服务容器，使用 CPR。在这个示例中，容器会执行一个预测请求和一个健康检查。

*注意:* 您需要在之前的步骤中设置好凭据，并在运行容器时传递凭据的路径。服务账号应具有 **Storage Object Admin** 权限。

In [ ]:
with local_model.deploy_to_local_endpoint(
    artifact_uri=f"{BUCKET_URI}/model",
    credential_path=CREDENTIALS_FILE,  # Update this to the path to your credentials.
) as local_endpoint:
    predict_response = local_endpoint.predict(
        request_file=INPUT_FILE,
        headers={"Content-Type": "application/json"},
    )

    health_check_response = local_endpoint.run_health_check()

打印预测响应及其内容。

In [ ]:
predict_response, predict_response.content

打印出健康检查的响应和内容。

In [ ]:
health_check_response, health_check_response.content

同时打印出所有容器日志。

In [ ]:
local_endpoint.print_container_logs(show_all=True)

### 将容器推送到工件存储库

#### 配置Docker以访问工件存储库

In [ ]:
! gcloud services enable artifactregistry.googleapis.com

In [ ]:
! gcloud beta artifacts repositories create {REPOSITORY} \
    --repository-format=docker \
    --location=$REGION

In [ ]:
! gcloud auth configure-docker {REGION}-docker.pkg.dev --quiet

将您的容器镜像推送到您的构件存储库。

In [ ]:
local_model.push_image()

将自定义模型服务容器部署到Vertex AI

将自定义服务容器上传到“Vertex AI模型”资源

使用LocalModel实例将自定义服务容器上传到“Vertex AI模型”资源。它将自动为您填充容器规范。

In [ ]:
model = aip.Model.upload(
    local_model=local_model,
    display_name="iris_" + TIMESTAMP,
    artifact_uri=f"{BUCKET_URI}/model",
)

将模型部署到 `Vertex AI Endpoint` 资源

接下来，将 Vertex AI 模型资源部署到 Vertex AI Endpoint 资源，用于预测。

In [ ]:
endpoint = model.deploy(machine_type="n1-standard-4")

使用部署模型进行预测

### 使用 Vertex AI SDK

首先，您可以使用 Vertex AI SDK 发出预测请求。

In [ ]:
endpoint.predict(instances=[[6.7, 3.1, 4.7, 1.5], [4.6, 3.1, 1.5, 0.2]])

使用REST

接下来，您重复相同的操作，但使用REST接口发出预测请求。

In [ ]:
ENDPOINT_ID = endpoint.name

In [ ]:
! curl \
-H "Authorization: Bearer $(gcloud auth print-access-token)" \
-H "Content-Type: application/json" \
-d @instances.json \
https://{REGION}-aiplatform.googleapis.com/v1/projects/{PROJECT_ID}/locations/{REGION}/endpoints/{ENDPOINT_ID}:predict

使用gcloud CLI

最后，您重复相同的操作，但是使用gcloud命令行界面来发出预测请求。

In [ ]:
! gcloud ai endpoints predict $ENDPOINT_ID \
  --region=$REGION \
  --json-request=instances.json

清理：情景1

In [ ]:
try:
    # Undeploy model and delete endpoint
    endpoint.delete(force=True)

    # Delete the model resource
    model.delete()
except:
    pass

# Delete the container image from Artifact Registry
! gcloud artifacts docker images delete \
    --quiet \
    --delete-tags \
    {REGION}-docker.pkg.dev/{PROJECT_ID}/{REPOSITORY}/{SERVER_IMAGE}

! rm -rf model

## 使用CPR模型服务器构建自定义模型服务容器：场景2：实现预测器

接下来，您将为CPR模型服务器实现自定义`predictor()`方法，而不是使用预先构建的预测器。`predictor()`方法处理将实例数据发送到模型并接收预测请求。它还将调用`preprocess()`方法对输入数据进行预处理，然后将其发送到模型。在此示例中，您继承基类`Predictor`，并实现相应的`predictor()`方法。

```python
class CprPredictor(Predictor):
    """用于Sklearn模型的默认预测器实现。"""

    def __init__(self):
        return

    def load(self, gcs_artifacts_uri: str):
        """加载模型工件。
           加载预处理器模块。
        """

    def preprocess(self, prediction_input: dict):
        """(super)将预处理器应用于输入数据。
        """

    def predict(self, instances: np.ndarray):
        """执行预测。
        """
```

In [ ]:
%%writefile src/predictor.py

import joblib
import numpy as np
import pickle

from google.cloud import storage
from google.cloud.aiplatform.prediction.predictor import Predictor

from sklearn.datasets import load_iris


class CprPredictor(Predictor):
    
    def __init__(self):
        return
    
    def load(self, gcs_artifacts_uri: str):
        """Loads the preprocessor and model artifacts."""
        gcs_client = storage.Client()
        with open("preprocessor.pkl", 'wb') as preprocessor_f, open("model.joblib", 'wb') as model_f:
            gcs_client.download_blob_to_file(
                f"{gcs_artifacts_uri}/preprocessor.pkl", preprocessor_f
            )
            gcs_client.download_blob_to_file(
                f"{gcs_artifacts_uri}/model.joblib", model_f
            )

        with open("preprocessor.pkl", "rb") as f:
            preprocessor = pickle.load(f)

        self._class_names = load_iris().target_names
        self._model = joblib.load("model.joblib")
        self._preprocessor = preprocessor

    def predict(self, instances):
        """Performs prediction."""
        instances = instances["instances"]
        inputs = np.asarray(instances)
        preprocessed_inputs = self._preprocessor.preprocess(inputs)
        outputs = self._model.predict(preprocessed_inputs)

        return {"predictions": [self._class_names[class_num] for class_num in outputs]}

## 构建自定义模型服务容器

接下来，您需要构建自定义模型的服务容器。

In [ ]:
import os

from google.cloud.aiplatform.prediction import LocalModel
from src.predictor import CprPredictor

local_model = LocalModel.build_cpr_model(
    "src",
    f"{REGION}-docker.pkg.dev/{PROJECT_ID}/{REPOSITORY}/{SERVER_IMAGE}",
    predictor=CprPredictor,
    requirements_path=os.path.join("src", "requirements.txt"),
)

获取用于服务容器的规格

接下来，显示您刚刚建立的自定义服务容器的规格。

In [ ]:
local_model.get_serving_container_spec()

### 在本地测试自定义模型服务容器

接下来，您可以在本地使用 CPR 测试您的自定义模型服务容器。在这个例子中，容器执行预测请求和健康检查。

*注意:* 您需要在之前的步骤中设置好凭证，并且在运行容器时传递凭证的路径。服务账号应该具有 **存储对象管理员** 权限。

In [ ]:
with local_model.deploy_to_local_endpoint(
    artifact_uri=f"{BUCKET_URI}/model",
    credential_path=CREDENTIALS_FILE,
) as local_endpoint:
    predict_response = local_endpoint.predict(
        request_file=INPUT_FILE,
        headers={"Content-Type": "application/json"},
    )

    health_check_response = local_endpoint.run_health_check()

打印出预测响应及其内容。

In [ ]:
predict_response, predict_response.content

打印出健康检查的响应及其内容。

In [ ]:
health_check_response, health_check_response.content

也打印出所有容器日志。

In [ ]:
local_endpoint.print_container_logs(show_all=True)

### 推送容器到工件注册表

#### 配置Docker以访问工件注册表

In [ ]:
! gcloud auth configure-docker {REGION}-docker.pkg.dev --quiet

将您的容器镜像推送到Artifact Registry存储库

In [ ]:
local_model.push_image()

将自定义模型Serving容器部署到Vertex AI

使用LocalModel实例将自定义Serving容器上传到“Vertex AI模型”资源。它会自动为您填充容器规范。

In [ ]:
model = aip.Model.upload(
    local_model=local_model,
    display_name="iris_" + TIMESTAMP,
    artifact_uri=f"{BUCKET_URI}/model",
)

### 将模型部署到 `Vertex AI Endpoint` 资源

接下来，部署 Vertex AI 模型资源到 Vertex AI Endpoint 资源，用于预测。

In [ ]:
endpoint = model.deploy(machine_type="n1-standard-4")

使用顶点 AI SDK 进行部署模型的预测

使用顶点 AI SDK 发送一个预测请求。

In [ ]:
endpoint.predict(instances=[[6.7, 3.1, 4.7, 1.5], [4.6, 3.1, 1.5, 0.2]])

清理：场景2

In [ ]:
try:
    # Undeploy model and delete endpoint
    endpoint.delete(force=True)

    # Delete the model resource
    model.delete()
except:
    pass

# Delete the container image from Artifact Registry
! gcloud artifacts docker images delete \
    --quiet \
    --delete-tags \
    {REGION}-docker.pkg.dev/{PROJECT_ID}/{REPOSITORY}/{SERVER_IMAGE}

! rm -rf model src/entrypoint.py

使用CPR模型服务器构建自定义模型服务容器：场景3：实现预测器和请求处理程序

接下来，您将为CPR模型服务器实现自定义的`handler()`方法，而不是使用预先构建的http请求处理程序。`handler()`方法用于处理从HTTP请求消息中提取预测请求。它还将调用`predictor()`方法，传递提取的实例数据以进行预测请求。

[`Handler`](https://github.com/googleapis/python-aiplatform/blob/custom-prediction-routine/google/cloud/aiplatform/prediction/handler.py)必须实现以下接口。

```
class CprHandler(PredictionHandler):
    """用于处理预测请求的Handler类的接口。"""

    def __init__(
        self, gcs_artifacts_uri: str, predictor: Optional[Type[Predictor]] = None,
    ):
        """初始化一个处理程序实例。
        Args:
            gcs_artifacts_uri (str):
                必填。环境变量AIP_STORAGE_URI的值。
            predictor (Type[Predictor]):
                可选。如果给定，则此处理程序使用的Predictor类以初始化预测器实例。
        """

    def handle(self, request: Request) -> Response:
        """处理预测请求。
        Args:
            request (Request):
                发送到应用程序的请求。
        Returns:
            预测请求的响应。
        """
```

In [ ]:
%%writefile src/handler.py

import csv
from io import StringIO
import json

from fastapi import Response

from google.cloud.aiplatform.prediction.handler import PredictionHandler

class CprHandler(PredictionHandler):
    """Default prediction handler for the prediction requests sent to the application."""

    async def handle(self, request):
        """Handles a prediction request."""
        request_body = await request.body()
        prediction_instances = self._convert_csv_to_list(request_body)
        prediction_instances = {"instances": prediction_instances}

        prediction_results = self._predictor.postprocess(
            self._predictor.predict(self._predictor.preprocess(prediction_instances))
        )

        return Response(content=json.dumps(prediction_results))
    
    def _convert_csv_to_list(self, data):
        """Converts list of string in csv format to list of float.
        
        Example input:
          b"1.1,2.2,3.3,4.4\n2.3,3.4,4.5,5.6\n"
          
        Example output:
            [
                [1.1, 2.2, 3.3, 4.4],
                [2.3, 3.4, 4.5, 5.6],
            ]
        """
        res = []
        for r in csv.reader(StringIO(data.decode("utf-8")), quoting=csv.QUOTE_NONNUMERIC):
            res.append(r)
        return res

## 构建自定义模型服务容器

接下来，您要构建自定义模型服务容器。

In [ ]:
import os

from google.cloud.aiplatform.prediction import LocalModel
from src.handler import CprHandler
from src.predictor import CprPredictor

local_model = LocalModel.build_cpr_model(
    "src",
    f"{REGION}-docker.pkg.dev/{PROJECT_ID}/{REPOSITORY}/{SERVER_IMAGE}",
    predictor=CprPredictor,
    handler=CprHandler,
    requirements_path=os.path.join("src", "requirements.txt"),
)

获取服务容器的规格

接下来，显示刚刚建立的自定义服务容器的规格。

In [ ]:
local_model.get_serving_container_spec()

### 创建示例数据

接下来，创建一些合成示例数据，并将这些示例存储在 CSV 格式中，用于预测。

要将输入实例发送到 CSV 中，需要使用 raw predict 来使用任意的 HTTP 负载，而不是 JSON 格式。

了解更多关于[Raw Predict](https://cloud.google.com/vertex-ai/docs/reference/rest/v1/projects.locations.endpoints/rawPredict)。

In [ ]:
INPUT_FILE = "instances.csv"

In [ ]:
%%writefile $INPUT_FILE
6.7,3.1,4.7,1.5
4.6,3.1,1.5,0.2

### 在本地测试自定义模型服务容器

接下来，您可以在本地使用 CPR 测试您的自定义模型服务容器。在这个例子中，容器执行一个预测请求和一个健康检查。

*注意:* 您需要在前一步设置好凭据，并在运行容器时传递凭据的路径。服务账号应该具有**存储对象管理员**权限。

In [ ]:
with local_model.deploy_to_local_endpoint(
    artifact_uri=f"{BUCKET_URI}/model",
    credential_path=CREDENTIALS_FILE,
) as local_endpoint:
    predict_response = local_endpoint.predict(
        request_file=INPUT_FILE,
        headers={"Content-Type": "application/json"},
    )

    health_check_response = local_endpoint.run_health_check()

打印出预测的响应及其内容。

In [ ]:
predict_response, predict_response.content

打印出健康检查的响应和内容。

In [ ]:
health_check_response, health_check_response.content

也打印出所有容器日志。

In [ ]:
local_endpoint.print_container_logs(show_all=True)

### 将容器推送至制品存储库

#### 配置Docker以访问制品存储库

In [ ]:
! gcloud auth configure-docker {REGION}-docker.pkg.dev --quiet

将您的容器镜像推送到Artifact Registry存储库

In [ ]:
local_model.push_image()

将自定义模型服务容器部署到 Vertex AI

### 将自定义服务容器上传到`Vertex AI模型`资源

使用LocalModel实例将自定义服务容器上传到`Vertex AI模型`资源。它会自动为您填充容器规范。

In [ ]:
model = aip.Model.upload(
    local_model=local_model,
    display_name="iris_" + TIMESTAMP,
    artifact_uri=f"{BUCKET_URI}/model",
)

### 部署模型到 `Vertex AI Endpoint` 资源

接下来，部署 Vertex AI 模型资源到一个 Vertex AI Endpoint 资源，用于预测。

In [ ]:
endpoint = model.deploy(machine_type="n1-standard-4")

## 对部署的模型进行预测

### 使用Vertex AI SDK

使用Vertex AI SDK的`rawPredict()`方法进行预测请求。

In [ ]:
from google.api import httpbody_pb2
from google.cloud import aiplatform_v1 as gapic

prediction_client = gapic.PredictionServiceClient(
    client_options={"api_endpoint": f"{REGION}-aiplatform.googleapis.com"}
)

with open(INPUT_FILE) as f:
    http_body = httpbody_pb2.HttpBody(
        data=f.read().encode("utf-8"),
        content_type="text/csv",
    )

request = gapic.RawPredictRequest(
    endpoint=endpoint.resource_name,
    http_body=http_body,
)

prediction_client.raw_predict(request=request)

清理：场景3

In [ ]:
try:
    # Undeploy model and delete endpoint
    endpoint.delete(force=True)

    # Delete the model resource
    model.delete()
except:
    pass

# Delete the container image from Artifact Registry
! gcloud artifacts docker images delete \
    --quiet \
    --delete-tags \
    {REGION}-docker.pkg.dev/{PROJECT_ID}/{REPOSITORY}/{SERVER_IMAGE}

! rm -rf model src/entrypoint.py

使用CPR模型服务器构建自定义模型服务容器：场景4：实现Docker构建过程

接下来，您将实现Docker构建过程，而不是使用预定义的Docker构建过程。

首先，编写容器的入口文件，该文件将启动自定义模型服务器。

In [ ]:
%%writefile src/entrypoint.py

import os
from typing import Optional, Type

from google.cloud.aiplatform import prediction

from predictor import CprPredictor
from handler import CprHandler


def main(
    predictor_class: Optional[Type[prediction.predictor.Predictor]] = None,
    handler_class: Type[prediction.handler.Handler] = prediction.handler.PredictionHandler,
    model_server_class: Type[prediction.model_server.ModelServer] = prediction.model_server.ModelServer,
):
    handler = handler_class(
        os.environ.get("AIP_STORAGE_URI"), predictor=predictor_class
    )

    return model_server_class(handler).start()

if __name__ == "__main__":
    main(
        predictor_class=CprPredictor,
        handler_class=CprHandler
    )

### 构建自定义模型服务容器

#### 编写 Docker 文件。

首先，构建 Docker 文件。 *注意：* 您需要将入口点指定为您定义的入口点模块。

In [ ]:
%%writefile Dockerfile

# Users select base images.
FROM python:3.7

# Sets the directories' permissions so that any user can access the folder.
RUN mkdir -m 777 -p /home /usr/app
ENV HOME=/home
WORKDIR /usr/app

# Copies all the stuff to the image.
COPY src /usr/app/src
COPY src/requirements.txt /usr/app/requirements.txt

# Installs python dependencies.
RUN pip3 install --no-cache-dir -r /usr/app/requirements.txt

# Informs Docker that the container listens on the specified ports at runtime.
EXPOSE 8080

# Sets up an entrypoint to start the model server.
ENTRYPOINT ["python3", "/usr/app/src/entrypoint.py"]

构建容器镜像

接下来，构建容器镜像。

In [ ]:
! docker build --tag={REGION}-docker.pkg.dev/{PROJECT_ID}/{REPOSITORY}/{SERVER_IMAGE} .

### 在本地测试自定义模型服务容器

接下来，您可以在本地使用 CPR 测试您的自定义模型服务容器。在这个例子中，容器执行一个预测请求和一个健康检查。

*注意：* 你需要在之前的步骤中设置好凭据，并在运行容器时传递凭据的路径。该服务账户应具有 **存储对象管理员** 权限。

In [ ]:
! docker run -d -p 80:8080 \
    --name=local-iris-custom \
    -e AIP_HTTP_PORT=8080 \
    -e AIP_HEALTH_ROUTE=/health \
    -e AIP_PREDICT_ROUTE=/predict \
    -e AIP_STORAGE_URI={BUCKET_URI}/model \
    -e GOOGLE_APPLICATION_CREDENTIALS=/usr/app/credentials.json \
    -e GOOGLE_CLOUD_PROJECT={PROJECT_ID} \
    -v {CREDENTIALS_FILE}:/usr/app/credentials.json \
    {REGION}-docker.pkg.dev/{PROJECT_ID}/{REPOSITORY}/{SERVER_IMAGE}

打印出健康检查的响应及其内容。

In [ ]:
! curl localhost/health

打印出预测的响应及其内容。

In [ ]:
! curl -X POST \
  -d @instances.csv \
  -H "Content-Type: application/json; charset=utf-8" \
  localhost/predict

关闭Docker服务

In [ ]:
! docker stop local-iris-custom

### 将容器推送到Artifact Registry

#### 配置Docker以访问Artifact Registry

In [ ]:
! gcloud auth configure-docker {REGION}-docker.pkg.dev --quiet

将您的容器映像推送到您的 Artifact Registry 存储库。

In [ ]:
local_model.push_image()

将自定义模型服务容器部署到Vertex AI

将自定义服务容器上传到`Vertex AI Model`资源

使用LocalModel实例将自定义服务容器上传到`Vertex AI Model`资源。它会自动为您填充容器规范。

In [ ]:
model = aip.Model.upload(
    local_model=local_model,
    display_name="iris_" + TIMESTAMP,
    artifact_uri=f"{BUCKET_URI}/model",
)

部署模型到`Vertex AI Endpoint`资源

接下来，将Vertex AI模型资源部署到Vertex AI Endpoint资源，用于预测。

In [ ]:
endpoint = model.deploy(machine_type="n1-standard-4")

## 对部署的模型进行预测

### 使用Vertex AI SDK

使用Vertex AI SDK的 `rawPredict()` 方法进行预测请求。

In [ ]:
from google.api import httpbody_pb2
from google.cloud import aiplatform_v1 as gapic

prediction_client = gapic.PredictionServiceClient(
    client_options={"api_endpoint": f"{REGION}-aiplatform.googleapis.com"}
)

with open(INPUT_FILE) as f:
    http_body = httpbody_pb2.HttpBody(
        data=f.read().encode("utf-8"),
        content_type="text/csv",
    )

request = gapic.RawPredictRequest(
    endpoint=endpoint.resource_name,
    http_body=http_body,
)

prediction_client.raw_predict(request=request)

清理：情景4

In [ ]:
try:
    # Undeploy model and delete endpoint
    endpoint.delete(force=True)

    # Delete the model resource
    model.delete()
except:
    pass

# Delete the container image from Artifact Registry
! gcloud artifacts docker images delete \
    --quiet \
    --delete-tags \
    {REGION}-docker.pkg.dev/{PROJECT_ID}/{REPOSITORY}/{SERVER_IMAGE}

! rm -rf model src/entrypoint.py

## 清理

要清理此项目中使用的所有Google Cloud资源，您可以[删除用于教程的Google Cloud项目](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects)。

否则，您可以删除在此教程中创建的各个资源。

In [ ]:
delete_bucket = False

if delete_bucket or os.getenv("IS_TESTING"):
    ! gsutil rm -rf {BUCKET_URI}

! rm -rf src model instances.json instances.csv Dockerfile